In [1]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [2]:
from geopy.geocoders import Nominatim
from pprint import pprint

app = Nominatim(user_agent='tutorial')
location = app.geocode('시화호')
pprint(location)

Location(시화호, 화성시, 경기도, 대한민국, (37.297262, 126.66935315697435, 0.0))


In [3]:
pip install pyproj

In [4]:
import pandas as pd
import numpy as np
import pyproj
import folium

In [5]:
df = pd.read_csv("./file/pig-location2.csv", encoding='cp949', usecols=['사업장명','좌표정보(x)','좌표정보(y)'],  header=0, engine='python')

In [6]:
type(df)
df

,사업장명,좌표정보(x),좌표정보(y)
0,경반축산,242825.1516,481055.1473
1,경반농장,243077.5124,480767.1901
2,골든피그하우스,238763.5401,478693.2070
3,자연농장,246541.6953,488525.2934
4,가평팜,229680.9439,478513.2232
...,...,...,...
1288,대림농장,194624.5664,402269.2594
1289,향남양돈조합법인,195773.3583,400291.3526
1290,주석농장,196805.8396,400610.0035
1291,재성농장,197692.8878,403827.1256


In [7]:
df['좌표정보(x)'] = pd.to_numeric(df['좌표정보(x)'], errors="coerce")
df['좌표정보(y)'] = pd.to_numeric(df['좌표정보(y)'], errors="coerce")

df = df.dropna()
df.index=range(len(df))
df.tail()

,사업장명,좌표정보(x),좌표정보(y)
1288,대림농장,194624.5664,402269.2594
1289,향남양돈조합법인,195773.3583,400291.3526
1290,주석농장,196805.8396,400610.0035
1291,재성농장,197692.8878,403827.1256
1292,병주농장,201302.4636,413153.8369


In [8]:
def project_array(coord, p1_type, p2_type):
    """
    좌표계 변환 함수
    - coord: x, y 좌표 정보가 담긴 NumPy Array
    - p1_type: 입력 좌표계 정보 ex) epsg:5179
    - p2_type: 출력 좌표계 정보 ex) epsg:4326
    """
    p1 = pyproj.Proj(init=p1_type)
    p2 = pyproj.Proj(init=p2_type)
    fx, fy = pyproj.transform(p1, p2, coord[:, 1], coord[:, 2])
    return np.dstack([fx, fy])[0]

In [9]:
# DataFrame -> NumPy Array 변환
coord = np.array(df)
coord

array([['경반축산', 242825.1516, 481055.1473],
       ['경반농장', 243077.5124, 480767.1901],
       ['골든피그하우스', 238763.5401, 478693.207],
       ...,
       ['주석농장', 196805.8396, 400610.0035],
       ['재성농장', 197692.8878, 403827.1256],
       ['병주농장', 201302.4636, 413153.8369]], dtype=object)

In [10]:
# 좌표계 정보 설정
p1_type = "epsg:2097"
p2_type = "epsg:4326"

# project_array() 함수 실행
result = project_array(coord, p1_type, p2_type)
result

C:\Users\SUSER\anaconda3\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\SUSER\anaconda3\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\SUSER\AppData\Local\Temp\ipykernel_18744\1770718179.py:10: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  fx, fy 

array([[127.48435364,  37.83106382],
       [127.48720305,  37.82845771],
       [127.43809222,  37.80996577],
       ...,
       [126.96196727,  37.10724616],
       [126.97193658,  37.13623692],
       [127.01257603,  37.22027726]])

In [11]:
df['경도'] = result[:, 0]
df['위도'] = result[:, 1]

In [12]:
df.tail()

,사업장명,좌표정보(x),좌표정보(y),경도,위도
1288,대림농장,194624.5664,402269.2594,126.937415,37.122187
1289,향남양돈조합법인,195773.3583,400291.3526,126.950353,37.104371
1290,주석농장,196805.8396,400610.0035,126.961967,37.107246
1291,재성농장,197692.8878,403827.1256,126.971937,37.136237
1292,병주농장,201302.4636,413153.8369,127.012576,37.220277


In [13]:
# 데이터 100개 랜덤 추출
sample = df.sample(n=100)

# 지도 중심 좌표 설정
lat_c, lon_c = 37.53165351203043, 126.9974246490573

# Folium 지도 객체 생성
m = folium.Map(location=[lat_c, lon_c], zoom_start=6)

# 마킹 표시 생성
for _, row in sample.iterrows():
    lat, lon  = row['위도'], row['경도']
    title = row['사업장명'] 
    folium.Marker(location=[lat, lon], tooltip = title).add_to(m)
###    folium.Marker([sub_lat,sub_long], tooltip = title).add_to(m)
    
m